In [3]:
import requests
import pandas as pd
import time
import os
from io import StringIO

import warnings
warnings.filterwarnings("ignore")

def truncate(sio):
    sio.truncate(0) # 清空第0個位置之後的資料
    sio.seek(0)  #保險一點多寫用seek從第0個位置開始添加
    return sio

save_file_path = 'my_stock'
    
if not os.path.exists(save_file_path):
        os.makedirs(save_file_path)
        
stock_list = ['2330']
start_month = '2021-01'
end_month = '2022-02'

month_range = [str(data).replace('-','').split(' ')[0] for data in pd.date_range(start = start_month, end = end_month, freq = 'MS')]

columns = ["日期","成交股數","成交金額","開盤價","最高價","最低價","收盤價","漲跌價差","成交筆數"]
my_df =pd.DataFrame(columns = columns)

for stock in stock_list: # 走過每支股票
    for m in month_range:  # 走過每個月份
        url = 'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date='+ m +'&stockNo='+stock
        print('processing:',url)
#         headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36'}
        headers = {'User-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
        res = requests.get(url, headers = headers)
        res.text

        io_csv_info = StringIO(res.text)
        io_csv_info.readline()  #讀取第一行(股票代號資訊我們不需要)
        io_csv_info.readline()  # 我們不需要 '"日期","成交股數","成交金額","開盤價","最高價","最低價","收盤價","漲跌價差","成交筆數",\r\n'

        # 接下來開始是需要的資訊，所以要用while迴圈處理
        while True:  # 走過每一天
            line = io_csv_info.readline()
            info_list = line.split('","')
            info_list[0] = info_list[0].strip('"')
            info_list[-1] = info_list[-1].strip('",\r\n')
        #     print(info_list)

            if '"說明:"\r\n' in line:  # 當讀到說明欄位，則跳出
                break

            df = pd.DataFrame([info_list],columns = columns) # 每一行建立df
            my_df = my_df.append(df,ignore_index = True)  # 把每一行的df 添加到mt_df。 ignore_index = True 最左邊的序號刪除
        truncate(io_csv_info) # 清除舊有的記憶體資料
        time.sleep(7)
    #走完一隻股票要去做存檔的動作:
    
#     my_df.to_csv(save_file_path + f'/{stock}_from_{start_month}_to_{end_month}.csv' , encoding = 'utf-8')
        my_df.to_csv(save_file_path + f'/{stock}.csv', encoding = 'utf-8-sig')
    print(f'=====================finished {stock} crawl========================================')

my_df

processing: https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date=20210101&stockNo=2330
processing: https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date=20210201&stockNo=2330
processing: https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date=20210301&stockNo=2330
processing: https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date=20210401&stockNo=2330
processing: https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date=20210501&stockNo=2330
processing: https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date=20210601&stockNo=2330
processing: https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date=20210701&stockNo=2330
processing: https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date=20210801&stockNo=2330
processing: https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date=20210901&stockNo=2330
processing: https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date=20211001&sto

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,110/01/04,"39,489,959","21,127,581,248",530.00,540.00,528.00,536.00,+6.00,"33,316"
1,110/01/05,"34,839,391","18,761,831,567",536.00,542.00,535.00,542.00,+6.00,"28,512"
2,110/01/06,"55,614,434","30,572,783,229",555.00,555.00,541.00,549.00,+7.00,"55,462"
3,110/01/07,"53,392,763","30,018,630,685",554.00,570.00,553.00,565.00,+16.00,"47,905"
4,110/01/08,"62,957,148","36,339,702,855",580.00,580.00,571.00,580.00,+15.00,"56,426"
...,...,...,...,...,...,...,...,...,...
272,111/02/21,"25,769,408","16,320,629,980",633.00,636.00,632.00,632.00,-5.00,"41,961"
273,111/02/22,"38,526,210","24,153,923,316",628.00,630.00,625.00,627.00,-5.00,"68,303"
274,111/02/23,"34,561,338","21,606,428,438",624.00,629.00,623.00,625.00,-2.00,"50,926"
275,111/02/24,"80,672,979","49,358,065,299",618.00,619.00,604.00,604.00,-21.00,"188,248"
